In [31]:
import logging
from functools import wraps

from telegram import (ReplyKeyboardMarkup, ReplyKeyboardRemove, ChatAction, ParseMode)
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters, RegexHandler,
                          ConversationHandler)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

In [3]:
def send_typing_action(func):
    
    @wraps(func)
    def command_func(bot, update, *args, **kwargs):
        bot.send_chat_action(chat_id=update.effective_message.chat_id, action=ChatAction.TYPING)
        return func(bot, update, *args, **kwargs)
        
    return command_func

In [89]:
cota_chats = {}

In [90]:
class Cota:
    def __init__(self, _id, name, value=None):
        self._id = _id
        self.name = name
        self.value = value
        self.going = []
        
class CotaView:
    def __init__(self, cota):
        self.cota = cota
        
    def simple_view(self):
        val = '' if not self.cota.value else ' - R$ {}'.format(self.cota.value)
        return '{} - {}'.format(self.cota._id, self.cota.name) + val

In [91]:
# All possible Interactive Boxes States

class MainListState:
    def __init__(self, iBox):
        self.iBox = iBox
        
    def generate_html(self):
        if not self.iBox.cota_chat.all_cotas:
            return '_Nenhuma cota encontrada!_'
        cota_views = [CotaView(cota) for cota in self.iBox.cota_chat.all_cotas.values()]
        return '\n'.join(['*Lista de Cotas:*'] + [cotaview.simple_view() for cotaview in cota_views])

In [92]:
class InteractiveBox:
    def __init__(self, cota_chat):
        self.message_id = None
        self.cota_chat = cota_chat
        
        self.current_state = MainListState(self)
        
    def generate_html(self):
        return self.current_state.generate_html()
    
    def update(self, bot):
        if not self.message_id:
            message = bot.send_message(self.cota_chat._id, "_Inicializando..._", parse_mode=ParseMode.MARKDOWN)
            self.message_id = message.message_id
        bot.edit_message_text(self.generate_html(), self.cota_chat._id, self.message_id, parse_mode=ParseMode.MARKDOWN)

class CotaChat:
    def __init__(self, _id):
        self._id = _id
        self.interactive_chat_boxes = []
        
        self.next_cota_id = 0
        self.all_cotas = {}
        
        self.temp_cota = None
        
    def new_interactive_box(self, bot):
        iBox = InteractiveBox(self)
        self.interactive_chat_boxes.append(iBox)
        iBox.update(bot)
        
    def update(self, bot):
        for icb in self.interactive_chat_boxes:
            icb.update(bot)
            
    def start_temp_cota(self, name):
        self.temp_cota = Cota(self.next_cota_id, name)
            
    def submit_temp_cota(self):
        self.all_cotas[self.temp_cota._id] = self.temp_cota
        self.temp_cota = None
        self.next_cota_id += 1
    

In [93]:
def get_cota_chat(update):
    chat_id = update.message.chat.id
    
    # Add this chat if not present
    if chat_id not in cota_chats:
        cota_chats[chat_id] = CotaChat(chat_id)
        
    return cota_chats[chat_id]

@send_typing_action
def cotabot(bot, update):
    cota_chat = get_cota_chat(update)
    cota_chat.new_interactive_box(bot)

@send_typing_action    
def new_cota(bot, update):
    cota_chat = get_cota_chat(update)
    bot.send_message(cota_chat._id, 'Ok! Qual o nome da cotinha? (/cancel para cancelar)')
    
    return 0

@send_typing_action
def cota_name(bot, update):
    cota_chat = get_cota_chat(update)
    cota_name = update.message.text
    
    cota_chat.start_temp_cota(cota_name)
    bot.send_message(cota_chat._id, 'Blz, qual o valor dela? (/skip pra colocar o valor depois, ou /cancel para cancelar)')
    
    return 1

def cota_value(bot, update):
    cota_chat = get_cota_chat(update)
    cota_value = float(update.message.text)
    
    cota_chat.temp_cota.value = cota_value
    submit_new_cota(bot, update)
    
@send_typing_action  
def submit_new_cota(bot, update):
    cota_chat = get_cota_chat(update)
    cota_chat.submit_temp_cota()
    
    bot.send_message(cota_chat._id, 'Cota criada!')
    cota_chat.update(bot)
    
    return ConversationHandler.END
    
    
def cancel_new_cota(bot, update):
    cota_chat = get_cota_chat(update)
    cota_chat.temp_cota = None
    
    bot.send_message(cota_chat._id, 'Criação de nova cota cancelada')
    
    return ConversationHandler.END

def error(bot, update, error):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, error)

In [94]:
def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("692336058:AAGFMBpvydprPwlYgQjwMM1QK66oH41qXfA")

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # add handlers
    dp.add_handler(CommandHandler('cotabot', cotabot))
    
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('newcota', new_cota)],
        states={0: [MessageHandler(Filters.text, cota_name)],
               1: [RegexHandler('\d+(.\d)?', cota_value),
                  CommandHandler('skip', submit_new_cota)]},
        fallbacks=[CommandHandler('cancel', cancel_new_cota)]
    )
    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
if __name__ == '__main__':
    main()